In [1]:
import casadi as ca
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [26]:
S = 2170.0
CBAR = 17.5
MASS = 5000
IYY = 4.1*10**6
TSTAT = 6.0*10**4
DTDV = -38.0
ZE = 2.0
CDCLS = 0.042
CLA = 0.085
CMA = -0.22
CMDE = -0.16
CMQ = -16.0
CMADOT= -6.0
CLADOT = 0
RTOD = 57.29578
GD = 32.17

In [9]:
THTL = ca.SX.sym('th')
ELEV = ca.SX.sym('el')
XCG = ca.SX.sym('xcg')
LAND = ca.SX.sym('ld')

u = ca.vertcat(THTL,ELEV,XCG,LAND)

VT = ca.SX.sym('vt')
ALPHA = RTOD * ca.SX.sym('a')
THETA = ca.SX.sym('theta')
Q = ca.SX.sym('q')
H = ca.SX.sym('h')

x = ca.vertcat(VT,ALPHA,THETA,Q,H)

In [13]:
#PAGE 634 FOR ADC IN FORTRAN
QBAR = ca.SX.sym('Qbar')
MACH = ca.SX.sym('mach')

In [16]:
QS = QBAR * S
SALP = ca.sin(x[1])
CALP = ca.cos(x[1])
GAM = THETA - x[1]
SGAM = ca.sin(GAM)
CGAM = ca.cos(GAM)

In [20]:
land = ca.SX.sym('land')

LAND0 = (0.20,0.16,0.05,0.0,0.0)
LAND1 = (1.0,0.08,-0.20,0.02,-0.05)

coeff = ca.if_else(land, LAND0, LAND1)
CLO = coeff[0]
CDO = coeff[1]
CMO = coeff[2]
DCDG = coeff[3]
DCMG = coeff[4]


In [21]:
THR = (TSTAT+VT*DTDV) * ca.fmax(THTL,0)
CL = CLO + CLA*ALPHA
CM = DCMG + CMO + CMA*ALPHA + CMDE*ELEV + CL* (XCG-0.25)
CD = DCDG + CDO + CDCLS*CL*CL

In [27]:
xd1 = (THR*CALP*QS*CD)/MASS - GD * SGAM
xd2 = (-THR*SALP-QS*CL+MASS*(VT*Q+GD+CGAM)) / (MASS*VT+QS*CLADOT)
xd3 = Q
D = 0.5*CBAR*(CMQ*Q+CMADOT*xd2)/VT
xd4 = (QS*CBAR*(CM + D) + THR*ZE)/IYY
xd5 = VT*SGAM
xd6 = VT*CGAM

In [28]:
xd = ca.vertcat(xd1,xd2,xd3,xd4,xd5,xd6)


In [29]:
from casadi.tools.graph import dotgraph
from IPython.display import Image

def draw_graph(expr):
    return Image(dotgraph(expr).create_png())

SX(@1=((60000+(-38*vt))*fmax(th,0)), @2=(57.2958*a), @3=(2170*Qbar), @4=(!land), @5=(((land?0.2:0)+(@4?1:0))+(0.085*@2)), @6=5000, @7=32.17, @8=(theta-@2), @9=sin(@8), @10=cos(@8), @11=(((@6*(((vt*q)+@7)+@10))-((@1*sin(@2))+(@3*@5)))/(@6*vt)), [(((((@1*cos(@2))*@3)*(((@4?0.02:0)+((land?0.16:0)+(@4?0.08:0)))+((0.042*@5)*@5)))/@6)-(@7*@9)), @11, q, ((((17.5*@3)*((((((@4?-0.05:0)+((land?0.05:0)+(@4?-0.2:0)))+(-0.22*@2))+(-0.16*el))+(@5*(xcg-0.25)))+((8.75*((-16*q)+(-6*@11)))/vt)))+(2*@1))/4.1e+06), (vt*@9), (vt*@10)])